In [ ]:
import pandas as pd
import numpy as np
import math
import pandas as pd
from pyhdf.SD import SD, SDC
import os
import math
import datetime as dt
import h5py

In [ ]:
GRANULES_PATH="ntl/VNP46A2/" #path to ntl data
OUTPUT_PATH="data/" #path to store csv files
LOCATION_FILE = 'geonames-all-cities-with-a-population-1000-3.csv' #path to cities data

In [ ]:
week_data={}
all_files=os.listdir(GRANULES_PATH)
for granule in all_files:
    if granule.endswith(".h5"):
        values=granule.split(".")
        year=int(values[1][1:5])
        date=dt.date(year,1,1)+dt.timedelta(days=int(values[1][5:8])-1)
        date_str=str(int(date.day))+"."+str(int(date.month))+"."+str(int(date.year))
        if date_str not in week_data:
            week_data[date_str]=[]
        week_data[date_str].append(granule)
all_weeks=[dt.date(int(date_str.split(".")[2]),int(date_str.split(".")[1]),int(date_str.split(".")[0])) for date_str in list(week_data.keys())]
all_weeks.sort()
all_weeks=[str(int(date.day))+"."+str(int(date.month))+"."+str(int(date.year)) for date in all_weeks]

In [ ]:
metrics={"Gap_Filled_DNB_BRDF-Corrected_NTL":"/HDFEOS/GRIDS/VNP_Grid_DNB/Data Fields/Gap_Filled_DNB_BRDF-Corrected_NTL"}

In [ ]:
degree=0.05

In [ ]:
fp=open(LOCATION_FILE,"r")
evrythng=fp.readlines()
fp.close()
locations=set()
for line in evrythng[1:]:
    line=line.split("\n")[0].split(";")
    [lat,long]=line[-1].split(",")
    if degree==1:
        lat=min(math.ceil(float(lat)),90.0)-0.5
        long=min(math.ceil(float(long)),180.0)-0.5
    elif degree==0.05:
        lat=round(90-(int((90-float(lat))/0.05)*0.05+0.025),3)
        long=round(180-(int((180-float(long))/0.05)*0.05+0.025),3)
    elif degree==0.25:
        lat=round(90-(int((90-float(lat))/0.25)*0.25+0.125),3)
        long=round(180-(int((180-float(long))/0.25)*0.25+0.125),3)
    else:
        print("wrong degree resolution provided!")
    locations.add((lat,long))

In [ ]:
def get_indices(location,granule):
    lat_tile_number=int(granule.split(".")[2].split("v")[1])
    long_tile_number=int(granule.split(".")[2].split("v")[0].split("h")[1])
    tile_lat_start_index=lat_tile_number*2400
    tile_lat_end_index=tile_lat_start_index+2400-1
    tile_long_start_index=long_tile_number*2400
    tile_long_end_index=tile_long_start_index+2400-1
    (lat,long)=location
    if degree==1:
        lat_start_index=int((90-lat-0.5)*240)
        lat_end_index=lat_start_index+240-1
        long_start_index=int((long-0.5+180)*240)
        long_end_index=long_start_index+240-1
    elif degree==0.05:
        lat_start_index=round(((90-lat-0.025)/0.05)*12)
        lat_end_index=lat_start_index+12-1
        long_start_index=round(((long-0.025+180)/0.05)*12)
        long_end_index=long_start_index+12-1
    elif degree==0.25:
        lat_start_index=round(((90-lat-0.125)/0.25)*60)
        lat_end_index=lat_start_index+60-1
        long_start_index=round(((long-0.125+180)/0.25)*60)
        long_end_index=long_start_index+60-1
    else:
        print("wrong degree resolution provided!")
    
    if lat_start_index>=tile_lat_start_index and lat_start_index<tile_lat_end_index:
        lat_range=(lat_start_index-tile_lat_start_index,lat_end_index-tile_lat_start_index)
    else:
        return None
    if long_start_index>=tile_long_start_index and long_start_index<tile_long_end_index:
        long_range=(long_start_index-tile_long_start_index,long_end_index-tile_long_start_index)
    else:
        return None
    return [lat_range,long_range]

In [ ]:
OUTPUT_FILENAME=OUTPUT_PATH+"indian_cities_daily_"+str(degree)+"degx"+str(degree)+"deg_ntl.csv"
fp=open(OUTPUT_FILENAME,"w")
fp.write(",".join(["location","week","metric","mean","sdev","min","10p","25p","median","75p","90p","max"])+"\n")
fp.close()
for week in all_weeks:
    print(week)
    #t1=dt.datetime.now()
    week_metric_data={}
    for metric in metrics:
        week_metric_data[metric]={}
    for granule in week_data[week]:
        f=h5py.File(GRANULES_PATH+granule, "r")
        for metric in metrics:
            metric_data=np.array(f[metrics[metric]])
            for location in locations:
                indices=get_indices(location,granule)
                if indices is None:
                    continue
                [lat_range,long_range]=indices
                temp=metric_data[lat_range[0]:lat_range[1]+1,long_range[0]:long_range[1]+1]
                temp=temp[np.where((temp>=0) & (temp<65535))]
                if len(temp)<=0:
                    continue
                if location not in week_metric_data[metric]:
                    week_metric_data[metric][location]=[]
                week_metric_data[metric][location]=np.append(week_metric_data[metric][location],temp)
    fp=open(OUTPUT_FILENAME,"a")
    for metric in week_metric_data:
        for location in week_metric_data[metric]:
            avg=round(np.mean(week_metric_data[metric][location]))
            sdev=round(np.std(week_metric_data[metric][location]))
            levels=[round(np.percentile(week_metric_data[metric][location],th)) for th in [0,10,25,50,75,90,100]]
            fp.write(",".join([str(location[0])+"#"+str(location[1]),week,metric]+[str(i) for i in [avg,sdev]+levels])+"\n")
    fp.close()
    #t2=dt.datetime.now()
    #print((t2-t1).seconds)

31.12.2018
1.1.2019
2.1.2019
3.1.2019
4.1.2019
5.1.2019
6.1.2019
7.1.2019
8.1.2019
9.1.2019
10.1.2019
11.1.2019
12.1.2019
13.1.2019
14.1.2019
15.1.2019
16.1.2019
17.1.2019
18.1.2019
19.1.2019
20.1.2019
22.1.2019
23.1.2019
24.1.2019
25.1.2019
26.1.2019
27.1.2019
28.1.2019
29.1.2019
30.1.2019
31.1.2019
1.2.2019
2.2.2019
3.2.2019
4.2.2019
5.2.2019
6.2.2019
7.2.2019
8.2.2019
9.2.2019
10.2.2019
11.2.2019
12.2.2019
13.2.2019
14.2.2019
15.2.2019
16.2.2019
17.2.2019
18.2.2019
19.2.2019
20.2.2019
21.2.2019
22.2.2019
23.2.2019
24.2.2019
25.2.2019
26.2.2019
27.2.2019
28.2.2019
1.3.2019
2.3.2019
3.3.2019
4.3.2019
5.3.2019
6.3.2019
7.3.2019
8.3.2019
9.3.2019
10.3.2019
11.3.2019
12.3.2019
13.3.2019
14.3.2019
15.3.2019
16.3.2019
17.3.2019
18.3.2019
19.3.2019
20.3.2019
21.3.2019
22.3.2019
23.3.2019
24.3.2019
25.3.2019
26.3.2019
27.3.2019
28.3.2019
29.3.2019
30.3.2019
31.3.2019
1.4.2019
2.4.2019
3.4.2019
4.4.2019
5.4.2019
6.4.2019
7.4.2019
8.4.2019
9.4.2019
10.4.2019
11.4.2019
12.4.2019
13.4.2019
14.4.

6.4.2021
7.4.2021
8.4.2021
9.4.2021
10.4.2021
11.4.2021
12.4.2021
13.4.2021
14.4.2021
15.4.2021
16.4.2021
17.4.2021
18.4.2021
19.4.2021
20.4.2021
21.4.2021
22.4.2021
23.4.2021
24.4.2021
25.4.2021
26.4.2021
27.4.2021
28.4.2021
29.4.2021
30.4.2021
1.5.2021
2.5.2021
3.5.2021
4.5.2021
5.5.2021
6.5.2021
7.5.2021
8.5.2021
9.5.2021
10.5.2021
11.5.2021
12.5.2021
13.5.2021
14.5.2021
15.5.2021
16.5.2021
17.5.2021
18.5.2021
19.5.2021
20.5.2021
21.5.2021
22.5.2021
23.5.2021
24.5.2021
25.5.2021
26.5.2021
27.5.2021
28.5.2021
29.5.2021
30.5.2021
31.5.2021
1.6.2021
2.6.2021
3.6.2021
4.6.2021
5.6.2021
6.6.2021
7.6.2021
8.6.2021
9.6.2021
10.6.2021
11.6.2021
12.6.2021
13.6.2021
14.6.2021
15.6.2021
16.6.2021
17.6.2021
18.6.2021
19.6.2021
20.6.2021
21.6.2021
22.6.2021
23.6.2021
24.6.2021
25.6.2021
26.6.2021
27.6.2021
28.6.2021
29.6.2021
30.6.2021
1.7.2021
2.7.2021
3.7.2021
4.7.2021
5.7.2021
6.7.2021
7.7.2021
8.7.2021
9.7.2021
10.7.2021
11.7.2021
12.7.2021
13.7.2021
14.7.2021
15.7.2021
16.7.2021
17.7.2021
1